In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data/finestapi_tracking_db.csv")
df

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata
0,4136,05/04/2018 19:18,Megastar,9773064,60.1469,24.9135,6.3,219,216,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
1,4137,05/04/2018 19:19,Megastar,9773064,60.1445,24.9100,11.6,217,217,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
2,4138,05/04/2018 19:20,Megastar,9773064,60.1412,24.9061,14.2,198,199,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
3,4139,05/04/2018 19:21,Star,9364722,59.4462,24.7726,3.7,17,159,EETLL,05/04/2018 19:30,07/04/2018 15:25,2018-04-05 19:21:17,FIHEL,05/04/2018 21:30,05/04/2018 21:26,NaN
4,4140,05/04/2018 19:22,Megastar,9773064,60.1344,24.9056,15.9,179,179,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823667,912179,15/03/2019 07:36,Finlandia,9214379,59.9242,24.8683,20.7,196,196,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,15/03/2019 09:02,NaN
823668,912180,15/03/2019 07:37,Finlandia,9214379,59.9198,24.8657,20.8,197,195,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,15/03/2019 09:02,NaN
823669,912181,15/03/2019 07:38,Finlandia,9214379,59.9148,24.8627,20.7,196,195,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,15/03/2019 09:02,NaN
823670,912182,15/03/2019 07:39,Finlandia,9214379,59.9082,24.8587,20.7,196,195,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,15/03/2019 09:02,NaN


In [ ]:
 df.describe()

,id,imo,lat,long,sog,cog,hdg
count,823672.000000,8.236720e+05,823672.000000,823672.000000,823672.000000,823672.000000,823672.000000
mean,462795.350583,9.401429e+06,59.800330,24.807672,19.285331,149.432205,156.911632
std,266273.511015,2.798543e+05,0.223849,0.084678,5.956900,111.171883,110.111634
min,4136.000000,8.919805e+06,59.420800,22.382100,0.000000,0.000000,0.000000
25%,228046.500000,9.214379e+06,59.602800,24.739800,16.400000,22.000000,23.000000
50%,463107.500000,9.364722e+06,59.799500,24.802200,21.900000,180.000000,183.000000
75%,696270.250000,9.773064e+06,60.000000,24.895600,22.800000,202.000000,204.000000
max,912183.000000,9.773064e+06,60.154000,24.984800,102.300000,360.000000,511.000000


In [ ]:
df["imo"].unique()

array([9773064, 9364722, 8919805, 9214379])

In [ ]:
MegastarDF = df[df["imo"] == 9773064]


In [ ]:
MegastarDF.describe()

,id,imo,lat,long,sog,cog,hdg
count,258132.000000,258132.0,258132.000000,258132.000000,258132.000000,258132.000000,258132.000000
mean,465259.047503,9773064.0,59.799162,24.807338,20.440860,145.074826,149.602901
std,265886.142359,0.0,0.222249,0.078926,5.538309,112.578630,110.441577
min,4136.000000,9773064.0,59.443800,24.598400,0.000000,0.000000,0.000000
25%,230640.500000,9773064.0,59.604200,24.740300,21.600000,18.000000,19.000000
50%,467135.500000,9773064.0,59.796900,24.800700,22.500000,180.000000,182.000000
75%,705966.500000,9773064.0,59.996900,24.895800,23.000000,200.000000,201.000000
max,912173.000000,9773064.0,60.148000,24.943200,28.900000,360.000000,511.000000


In [ ]:
MegastarFinlandDF = MegastarDF[MegastarDF["depPort"] == "FIHEL"]
MegastarEstoniaDF = MegastarDF[MegastarDF["depPort"] == "EETLL"]




1.  Convert Index to timeseries
2.  Create a new arrival time by substracting arrival time from updated
3.  Integrate with weather data










In [ ]:
MegastarFinlandDF = MegastarFinlandDF.dropna(thresh=16)


In [ ]:
MegastarFinlandDF.dtypes
MegastarFinlandDF.reset_index(drop=True)

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata
0,4136,05/04/2018 19:18,Megastar,9773064,60.1469,24.9135,6.3,219,216,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
1,4137,05/04/2018 19:19,Megastar,9773064,60.1445,24.9100,11.6,217,217,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
2,4138,05/04/2018 19:20,Megastar,9773064,60.1412,24.9061,14.2,198,199,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
3,4140,05/04/2018 19:22,Megastar,9773064,60.1344,24.9056,15.9,179,179,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
4,4142,05/04/2018 19:22,Megastar,9773064,60.1322,24.9056,16.1,181,181,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,05/04/2018 21:25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125934,911720,14/03/2019 22:22,Megastar,9773064,59.4501,24.7776,8.1,203,203,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,14/03/2019 22:30,14/03/2019 22:26,NaN
125935,911722,14/03/2019 22:23,Megastar,9773064,59.4484,24.7759,6.1,210,207,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,14/03/2019 22:30,14/03/2019 22:27,NaN
125936,911724,14/03/2019 22:24,Megastar,9773064,59.4471,24.7743,4.9,215,212,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,14/03/2019 22:30,14/03/2019 22:27,NaN
125937,911726,14/03/2019 22:24,Megastar,9773064,59.4463,24.7731,4.3,218,213,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,14/03/2019 22:30,14/03/2019 22:27,NaN


In [ ]:

!pip install statsmodels --upgrade
!pip install scipy --upgrade


Requirement already up-to-date: statsmodels in /usr/local/lib/python3.7/dist-packages (0.12.2)
Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (1.6.3)


In [ ]:
from datetime import datetime
MegastarFinlandDF["updated"] = pd.to_datetime(MegastarFinlandDF["updated"], format='%d/%m/%Y %H:%M')
MegastarFinlandDF["ata"] = pd.to_datetime(MegastarFinlandDF["ata"], format='%Y-%m-%d %H:%M:%S')
MegastarFinlandDF["etaSchedule"] = pd.to_datetime(MegastarFinlandDF["etaSchedule"], format='%d/%m/%Y %H:%M')
MegastarFinlandDF["eta"] = pd.to_datetime(MegastarFinlandDF["eta"], format='%d/%m/%Y %H:%M')
MegastarFinlandDF = MegastarFinlandDF.set_index('updated', drop = False)


In [ ]:
ArrivalIndex = MegastarFinlandDF[MegastarFinlandDF["ata"].notna()]
#ArrivalIndex.index
ArrivalIndex["ata"]
MegastarFinlandDF["timeToDestination"] = MegastarFinlandDF["ata"].bfill()


In [ ]:

MegastarFinlandDF["newTimeToDestination"] = MegastarFinlandDF["timeToDestination"] - MegastarFinlandDF["updated"]


In [ ]:
MegastarFinlandDF['newTimeToDestination'] = (MegastarFinlandDF['newTimeToDestination'] / np.timedelta64(1, 'm')).astype('int16')
MegastarFinlandDF['maximumJourneyDuration'] = ((MegastarFinlandDF["timeToDestination"] - MegastarFinlandDF["etaSchedule"]) / np.timedelta64(1, 'm')).astype('int16')
MegastarFinlandDF['etaError'] = ((MegastarFinlandDF["timeToDestination"] - MegastarFinlandDF["eta"]) / np.timedelta64(1, 'm')).astype('int16')

MegastarFinlandDF["forecastedTimeToDestination"] = MegastarFinlandDF["eta"] - MegastarFinlandDF["updated"]
MegastarFinlandDF['forecastedTimeToDestination'] = (MegastarFinlandDF['forecastedTimeToDestination'] / np.timedelta64(1, 'm')).astype('int16')

MegastarFinlandDF

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-04-05 19:18:00,4136,2018-04-05 19:18:00,Megastar,9773064,60.1469,24.9135,6.3,219,216,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,2018-04-05 21:30:00,2018-04-05 21:25:00,NaT,2018-04-05 21:24:41,126,-5,0,127
2018-04-05 19:19:00,4137,2018-04-05 19:19:00,Megastar,9773064,60.1445,24.9100,11.6,217,217,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,2018-04-05 21:30:00,2018-04-05 21:25:00,NaT,2018-04-05 21:24:41,125,-5,0,126
2018-04-05 19:20:00,4138,2018-04-05 19:20:00,Megastar,9773064,60.1412,24.9061,14.2,198,199,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,2018-04-05 21:30:00,2018-04-05 21:25:00,NaT,2018-04-05 21:24:41,124,-5,0,125
2018-04-05 19:22:00,4140,2018-04-05 19:22:00,Megastar,9773064,60.1344,24.9056,15.9,179,179,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,2018-04-05 21:30:00,2018-04-05 21:25:00,NaT,2018-04-05 21:24:41,122,-5,0,123
2018-04-05 19:22:00,4142,2018-04-05 19:22:00,Megastar,9773064,60.1322,24.9056,16.1,181,181,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,2018-04-05 21:30:00,2018-04-05 21:25:00,NaT,2018-04-05 21:24:41,122,-5,0,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-14 22:22:00,911720,2019-03-14 22:22:00,Megastar,9773064,59.4501,24.7776,8.1,203,203,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,2019-03-14 22:30:00,2019-03-14 22:26:00,NaT,2019-03-14 22:26:05,4,-3,0,4
2019-03-14 22:23:00,911722,2019-03-14 22:23:00,Megastar,9773064,59.4484,24.7759,6.1,210,207,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,2019-03-14 22:30:00,2019-03-14 22:27:00,NaT,2019-03-14 22:26:05,3,-3,0,4
2019-03-14 22:24:00,911724,2019-03-14 22:24:00,Megastar,9773064,59.4471,24.7743,4.9,215,212,FIHEL,14/03/2019 20:30,14/03/2019 20:30,2019-03-14 20:18:13,EETLL,2019-03-14 22:30:00,2019-03-14 22:27:00,NaT,2019-03-14 22:26:05,2,-3,0,3


In [ ]:

MegastarFinlandDF['maximumJourneyDuration'].describe()

count    125939.000000
mean         -0.740581
std          53.850868
min         -11.000000
25%          -5.000000
50%          -3.000000
75%          -2.000000
max        1440.000000
Name: maximumJourneyDuration, dtype: float64

In [ ]:
MegastarFinlandDF[(MegastarFinlandDF["maximumJourneyDuration"] > 0)]["maximumJourneyDuration"].unique()

array([   1,  714, 1436,   13,   12,   28,  775,  715,    2,   10,  157,
          9,  742,   22,   21,  718, 1437, 1440,   34,    3,    4],
      dtype=int16)

From further analysis we can see that delay greater than 34 minutes is an anomaly cause by missing data and data with 21 minute delay, has no prior history as only the arrival time is recorded

In [ ]:
#MegastarFinlandDF['2018-07-20'][MegastarFinlandDF['2018-07-20']["newTimeToDestination"] > 120]
#MegastarFinlandDF[(MegastarFinlandDF["newTimeToDestination"] > 170) & (MegastarFinlandDF["newTimeToDestination"] < 200)]
#[(MegastarFinlandDF["maximumJourneyDuration"] > 100) & (MegastarFinlandDF["maximumJourneyDuration"] < 200)]
#MegastarFinlandDF[(MegastarFinlandDF["maximumJourneyDuration"] > 11) & (MegastarFinlandDF["maximumJourneyDuration"] < 13)]
cleanedData = MegastarFinlandDF[(MegastarFinlandDF["maximumJourneyDuration"] < 35)& (MegastarFinlandDF["maximumJourneyDuration"] != 21)]
cleanedData.describe()

,id,imo,lat,long,sog,cog,hdg,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
count,125520.000000,125520.0,125520.000000,125520.000000,125520.000000,125520.000000,125520.000000,125520.000000,125520.000000,125520.000000,125520.000000
mean,468221.440511,9773064.0,59.790679,24.798845,21.025306,186.600613,186.921200,62.551418,-3.417559,-1.336496,64.575765
std,265342.569826,0.0,0.216292,0.081769,4.334317,16.873740,16.911885,36.601105,2.977347,3.401425,37.600269
min,4136.000000,9773064.0,59.444100,24.652700,0.000000,1.000000,144.000000,0.000000,-11.000000,-25.000000,-39.000000
25%,234471.500000,9773064.0,59.602000,24.726600,21.700000,179.000000,179.000000,31.000000,-5.000000,-3.000000,32.000000
50%,473692.000000,9773064.0,59.788000,24.785800,22.400000,191.000000,191.000000,62.000000,-3.000000,-1.000000,64.000000
75%,707034.500000,9773064.0,59.979900,24.891700,22.900000,199.000000,199.000000,94.000000,-2.000000,0.000000,96.000000
max,911728.000000,9773064.0,60.148000,24.943200,28.100000,343.000000,511.000000,176.000000,34.000000,48.000000,136.000000


In [ ]:
cleanedData[(cleanedData["etaError"] > 0)]["etaError"].unique()

array([ 1,  2,  3,  4, 10, 12, 11, 18,  5,  7,  6, 13, 25, 27, 26, 28, 29,
       30,  9, 14, 15, 42,  8, 22, 38, 39, 40, 48, 47], dtype=int16)

In [ ]:
etaErrorCount = len(cleanedData[(cleanedData["etaError"] >= 5)].index)
totalCleanDataCount = len(cleanedData.index)

import plotly.graph_objects as go

labels = ['Falsely Predicted_ETA','Total']
values = [etaErrorCount, totalCleanDataCount]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [ ]:
cleanedData[(cleanedData["etaError"] == 30)]

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-08-01 20:38:00,306550,2018-08-01 20:38:00,Megastar,9773064,59.7073,24.7008,22.0,192,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,80,28,30,50
2018-08-01 20:38:00,306553,2018-08-01 20:38:00,Megastar,9773064,59.7054,24.7000,22.0,192,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,80,28,30,50
2018-08-01 20:39:00,306556,2018-08-01 20:39:00,Megastar,9773064,59.6988,24.6973,22.0,192,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,79,28,30,49
2018-08-01 20:40:00,306559,2018-08-01 20:40:00,Megastar,9773064,59.6924,24.6946,21.9,192,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,78,28,30,48
2018-08-01 20:41:00,306562,2018-08-01 20:41:00,Megastar,9773064,59.6864,24.6922,21.9,192,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,77,28,30,47
2018-08-01 20:42:00,306565,2018-08-01 20:42:00,Megastar,9773064,59.6798,24.6895,21.9,192,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,76,28,30,46
2018-08-01 20:44:00,306568,2018-08-01 20:44:00,Megastar,9773064,59.6734,24.6869,22.0,191,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,74,28,30,44
2018-08-01 20:45:00,306571,2018-08-01 20:45:00,Megastar,9773064,59.6667,24.6843,22.0,191,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,73,28,30,43
2018-08-01 20:47:00,306576,2018-08-01 20:47:00,Megastar,9773064,59.6553,24.6799,22.1,191,191,FIHEL,01/08/2018 19:30,01/08/2018 19:30,2018-08-01 19:19:14,EETLL,2018-08-01 21:30:00,2018-08-01 21:28:00,NaT,2018-08-01 21:58:01,71,28,30,41


In [ ]:
cleanedData[cleanedData["hdg"] > 360]
# Heading should never be greater than 360

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-09-20 14:35:00,430608,2018-09-20 14:35:00,Megastar,9773064,59.7172,24.7060,22.3,190,511,FIHEL,20/09/2018 13:30,20/09/2018 13:30,2018-09-20 13:21:17,EETLL,2018-09-20 15:30:00,2018-09-20 15:27:00,NaT,2018-09-20 15:24:40,49,-5,-2,52
2018-09-20 14:35:00,430610,2018-09-20 14:35:00,Megastar,9773064,59.7117,24.7041,22.3,189,511,FIHEL,20/09/2018 13:30,20/09/2018 13:30,2018-09-20 13:21:17,EETLL,2018-09-20 15:30:00,2018-09-20 15:26:00,NaT,2018-09-20 15:24:40,49,-5,-1,51


In [ ]:
cleanedData["2018-09-20 14:35:00"]["hdg"] = 190
cleanedData["2018-09-20 14:35:00"]
# Changed the heading to some

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-09-20 14:35:00,430608,2018-09-20 14:35:00,Megastar,9773064,59.7172,24.7060,22.3,190,190,FIHEL,20/09/2018 13:30,20/09/2018 13:30,2018-09-20 13:21:17,EETLL,2018-09-20 15:30:00,2018-09-20 15:27:00,NaT,2018-09-20 15:24:40,49,-5,-2,52
2018-09-20 14:35:00,430610,2018-09-20 14:35:00,Megastar,9773064,59.7117,24.7041,22.3,189,190,FIHEL,20/09/2018 13:30,20/09/2018 13:30,2018-09-20 13:21:17,EETLL,2018-09-20 15:30:00,2018-09-20 15:26:00,NaT,2018-09-20 15:24:40,49,-5,-1,51


In [ ]:
cleanedData = cleanedData[~cleanedData.index.duplicated(keep='first')]


In [ ]:
x = cleanedData[["lat", "long", "sog", "cog", "hdg","forecastedTimeToDestination","newTimeToDestination"]].reset_index(drop=True)
x = x.astype({'lat': 'float32', 'long': 'float32', 'sog': 'float32', 'cog': 'float32','hdg': 'int32', 'newTimeToDestination': 'int32'})
x.dtypes


lat                            float32
long                           float32
sog                            float32
cog                            float32
hdg                              int32
forecastedTimeToDestination      int16
newTimeToDestination             int32
dtype: object

In [ ]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
scaler = StandardScaler().fit_transform(x)
# fit and transform the data
df_norm = pd.DataFrame(scaler, columns=x.columns)

df_norm

,lat,long,sog,cog,hdg,forecastedTimeToDestination,newTimeToDestination
0,1.649069,1.404821,-3.405449,1.921834,1.725334,1.657809,1.731713
1,1.637971,1.362069,-2.178040,1.803332,1.784627,1.631246,1.704412
2,1.622709,1.314424,-1.575915,0.677561,0.717341,1.604682,1.677112
3,1.591250,1.308320,-1.182218,-0.448209,-0.468533,1.551554,1.622512
4,1.557957,1.307108,-0.927473,-0.329707,-0.349945,1.524991,1.595212
...,...,...,...,...,...,...,...
107678,-1.562219,-0.236867,-2.340151,0.085051,0.065111,-1.582969,-1.571605
107679,-1.573775,-0.255202,-2.988593,0.973817,0.954516,-1.609533,-1.598905
107680,-1.581644,-0.275961,-3.451766,1.388574,1.191690,-1.609533,-1.626205
107681,-1.587643,-0.295508,-3.729670,1.684829,1.488159,-1.636097,-1.653505


In [ ]:
X = x[["lat", "long", "sog", "cog", "hdg"]]
y = x["newTimeToDestination"]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

regressor = RandomForestRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y)
x.dtypes

lat                            float32
long                           float32
sog                            float32
cog                            float32
hdg                              int32
forecastedTimeToDestination      int16
newTimeToDestination             int32
dtype: object

In [ ]:



regModel = regressor.fit(X_train,y_train)
regModel.score(X_test,y_test)


0.9944564197192254

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
x_data = np.array(x[["lat", "long", "sog", "cog", "hdg"]])

keras_normalizer = preprocessing.Normalization(input_shape=[5,])
keras_normalizer.adapt(x_data)
tf.keras.backend.floatx()

'float32'

In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras import backend as K

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def build_model():
  model = keras.models.Sequential()
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dense(64, activation='relu',input_shape=(X_train.shape[1],)))
  model.add(keras.layers.Dense(64, activation='relu'))
  model.add(keras.layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=[coeff_determination])
  return model

model = build_model()
model.fit(X_train, y_train,epochs=50, batch_size=1000)


Epoch 1/50
81/81 [==============================] - 1s 3ms/step - loss: 4886.9849 - coeff_determination: -2.6512
Epoch 2/50
81/81 [==============================] - 0s 3ms/step - loss: 2360.0213 - coeff_determination: -0.7612
Epoch 3/50
81/81 [==============================] - 0s 3ms/step - loss: 233.0664 - coeff_determination: 0.8263
Epoch 4/50
81/81 [==============================] - 0s 3ms/step - loss: 47.0156 - coeff_determination: 0.9649
Epoch 5/50
81/81 [==============================] - 0s 3ms/step - loss: 32.8240 - coeff_determination: 0.9754
Epoch 6/50
81/81 [==============================] - 0s 3ms/step - loss: 23.3187 - coeff_determination: 0.9827
Epoch 7/50
81/81 [==============================] - 0s 3ms/step - loss: 18.8631 - coeff_determination: 0.9859
Epoch 8/50
81/81 [==============================] - 0s 3ms/step - loss: 16.8155 - coeff_determination: 0.9875
Epoch 9/50
81/81 [==============================] - 0s 3ms/step - loss: 14.6293 - coeff_determination: 0.9891
Epo

In [ ]:
textDF = x.iloc[X_test.index]
textDF

,lat,long,sog,cog,hdg,forecastedTimeToDestination,newTimeToDestination
14620,59.745899,24.781099,22.799999,194.0,194,59,55
28224,59.519402,24.728201,22.900000,160.0,158,18,16
24375,59.609299,24.666201,21.500000,177.0,178,33,32
31983,60.097698,24.900600,18.799999,185.0,185,119,115
34581,59.910999,24.856100,22.900000,198.0,199,86,82
...,...,...,...,...,...,...,...
93964,59.633900,24.684500,21.500000,209.0,211,37,37
10360,60.067001,24.894899,22.100000,177.0,177,113,103
57492,60.077900,24.898001,21.799999,183.0,184,115,112
103919,59.449799,24.777300,8.600000,200.0,200,4,2


In [ ]:
from sklearn.metrics import r2_score
r2_score(textDF['newTimeToDestination'], textDF['forecastedTimeToDestination'])

0.9870245243492161

In [ ]:
testDataPrediction = regModel.predict(textDF[["lat", "long", "sog", "cog", "hdg"]])
testDataPrediction

array([ 55.58,  16.42,  32.78, ..., 110.81,   2.89,  46.7 ])

In [ ]:
neuralPrediction = model.predict(textDF[["lat", "long", "sog", "cog", "hdg"]])
neuralPrediction = [num[0] for num in neuralPrediction.tolist()]
neuralPrediction

In [ ]:
r2_score(y_test, neuralPrediction)

0.9932860284888829

In [ ]:
from sklearn.metrics import max_error
max_error(textDF['newTimeToDestination'], textDF['forecastedTimeToDestination'])

40

In [ ]:
max_error(y_test, testDataPrediction)

41.72

In [ ]:
max_error(y_test, neuralPrediction)

43.35503387451172

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(textDF['newTimeToDestination'], textDF['forecastedTimeToDestination'])

2.803016232680807

In [ ]:
mean_absolute_error(y_test, testDataPrediction)

1.4882840232709107

In [ ]:
mean_absolute_error(y_test, neuralPrediction)

1.8580291248183518

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=neuralPrediction[0:50],
                    mode='lines',
                    name='Model Predicted ETA'))
fig.add_trace(go.Scatter(y=y_test[0:50],
                    mode='lines',
                    fill='tozeroy',
                    name='True ETA'))

fig.add_trace(go.Scatter(y=textDF['forecastedTimeToDestination'][0:50],
                    mode='lines',
                    name= 'Data Calculated ETA'))

fig.show()

In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "helsinkiTripETA")

In [ ]:
from tensorflow.keras.models import Model


In [ ]:
input_img = keras.Input(shape=(5,))

encoder = tf.keras.Sequential([
  keras.layers.BatchNormalization(),
  keras.layers.Dense(4, activation="relu"),
  keras.layers.Dense(2, activation="relu")])(input_img)

decoder = tf.keras.Sequential([
  keras.layers.Dense(2, activation="relu"),
  keras.layers.Dense(5)])(encoder)

autoencoder = keras.Model(input_img, decoder)
  
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
sequential_15 (Sequential)   (None, 2)                 54        
_________________________________________________________________
sequential_16 (Sequential)   (None, 5)                 21        
Total params: 75
Trainable params: 65
Non-trainable params: 10
_________________________________________________________________


In [ ]:
history = autoencoder.fit(X_train, X_train, 
          epochs=50, 
          batch_size=100,
          validation_data=(X_test, X_test),
          shuffle=True)

Epoch 1/50
808/808 [==============================] - 2s 2ms/step - loss: 84.2709 - val_loss: 6.6473
Epoch 2/50
808/808 [==============================] - 1s 1ms/step - loss: 3.6685 - val_loss: 2.5353
Epoch 3/50
808/808 [==============================] - 1s 1ms/step - loss: 2.5284 - val_loss: 2.1994
Epoch 4/50
808/808 [==============================] - 1s 1ms/step - loss: 2.4744 - val_loss: 2.0910
Epoch 5/50
808/808 [==============================] - 1s 1ms/step - loss: 2.4347 - val_loss: 2.2937
Epoch 6/50
808/808 [==============================] - 1s 1ms/step - loss: 2.4063 - val_loss: 2.0067
Epoch 7/50
808/808 [==============================] - 1s 1ms/step - loss: 2.3760 - val_loss: 2.3432
Epoch 8/50
808/808 [==============================] - 1s 1ms/step - loss: 2.4211 - val_loss: 2.0431
Epoch 9/50
808/808 [==============================] - 1s 1ms/step - loss: 2.3580 - val_loss: 2.1308
Epoch 10/50
808/808 [==============================] - 1s 1ms/step - loss: 2.3201 - val_loss: 1.961

In [ ]:
reconstructions = autoencoder.predict(X_train)
train_loss = tf.keras.losses.mae(reconstructions, X_train)
train_loss

<tf.Tensor: shape=(80762,), dtype=float64, numpy=
array([1.9296032 , 1.82587357, 1.73206291, ..., 0.88778687, 1.65032616,
       1.40794907])>

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Threshold:  2.5071659276896954


In [ ]:
#reconstructions_est = autoencoder.predict(cleanedDataEstoniaLateArrival[["lat", "long", "sog", "cog", "hdg"]].reset_index(drop=True))
#test_loss = tf.keras.losses.mae(reconstructions_est, cleanedDataEstoniaLateArrival[["lat", "long", "sog", "cog", "hdg"]])
#threshold_est = np.mean(test_loss)
#threshold_est

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(autoencoder, "helsinkiTripHealth")

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

clf = OneClassSVM(nu=0.001)
clf.fit(x[["lat", "long", "sog", "cog", "hdg"]])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)

In [ ]:
outlierResult = clf.predict(cleanedDataEstoniaLateArrival[["lat", "long", "sog", "cog", "hdg"]].reset_index(drop=True))
newlist = [item for item in outlierResult if item == -1]

print(len(newlist), len(outlierResult))

NameError: ignored

In [ ]:
initial_type = [('x', FloatTensorType([None, x[["lat", "long", "sog", "cog", "hdg"]].shape[1]]))]
onx = convert_sklearn(clf, initial_types=initial_type)

with open("helsinkiNovelty.onnx", "wb") as f:
    f.write(onx.SerializeToString())

Plot the difference between the prediction and the initial eta calculated from the data

In [ ]:
MegastarEstoniaDF.describe()

,id,imo,lat,long,sog,cog,hdg
count,131108.000000,131108.0,131108.000000,131108.000000,131108.000000,131108.000000,131108.000000
mean,462802.231893,9773064.0,59.804224,24.814502,20.044025,104.812879,113.267543
std,266740.560642,0.0,0.226280,0.075028,6.215138,146.128513,145.096366
min,4376.000000,9773064.0,59.443800,24.598400,0.000000,0.000000,0.000000
25%,226488.000000,9773064.0,59.604600,24.752600,21.300000,11.000000,11.000000
50%,461511.500000,9773064.0,59.802500,24.812800,22.500000,18.000000,19.000000
75%,705823.750000,9773064.0,60.008600,24.896500,23.100000,339.000000,337.000000
max,912171.000000,9773064.0,60.148000,24.924000,28.900000,360.000000,511.000000


In [ ]:
MegastarEstoniaDF = MegastarEstoniaDF.dropna(thresh=16)
MegastarEstoniaDF.reset_index(drop=True)
MegastarEstoniaDF["updated"] = pd.to_datetime(MegastarEstoniaDF["updated"], format='%d/%m/%Y %H:%M')
MegastarEstoniaDF["ata"] = pd.to_datetime(MegastarEstoniaDF["ata"], format='%Y-%m-%d %H:%M:%S')
MegastarEstoniaDF["etaSchedule"] = pd.to_datetime(MegastarEstoniaDF["etaSchedule"], format='%d/%m/%Y %H:%M')
MegastarEstoniaDF["eta"] = pd.to_datetime(MegastarEstoniaDF["eta"], format='%d/%m/%Y %H:%M')
MegastarEstoniaDF = MegastarEstoniaDF.set_index('updated', drop = False)

ArrivalIndex = MegastarEstoniaDF[MegastarEstoniaDF["ata"].notna()]

MegastarEstoniaDF["timeToDestination"] = MegastarEstoniaDF["ata"].bfill()

MegastarEstoniaDF["newTimeToDestination"] = MegastarEstoniaDF["timeToDestination"] - MegastarEstoniaDF["updated"]

MegastarEstoniaDF['newTimeToDestination'] = (MegastarEstoniaDF['newTimeToDestination'] / np.timedelta64(1, 'm')).astype('int16')
MegastarEstoniaDF['maximumJourneyDuration'] = ((MegastarEstoniaDF["timeToDestination"] - MegastarEstoniaDF["etaSchedule"]) / np.timedelta64(1, 'm')).astype('int16')
MegastarEstoniaDF['etaError'] = ((MegastarEstoniaDF["timeToDestination"] - MegastarEstoniaDF["eta"]) / np.timedelta64(1, 'm')).astype('int16')

MegastarEstoniaDF["forecastedTimeToDestination"] = MegastarEstoniaDF["eta"] - MegastarEstoniaDF["updated"]
MegastarEstoniaDF['forecastedTimeToDestination'] = (MegastarEstoniaDF['forecastedTimeToDestination'] / np.timedelta64(1, 'm')).astype('int16')

MegastarEstoniaDF

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-04-06 04:18:00,4381,2018-04-06 04:18:00,Megastar,9773064,59.4459,24.7726,3.6,34,175,EETLL,06/04/2018 04:30,06/04/2018 04:30,2018-04-06 04:18:17,FIHEL,2018-04-06 06:30:00,2018-04-06 06:23:00,NaT,2018-04-06 06:27:09,129,-2,4,125
2018-04-06 04:19:00,4382,2018-04-06 04:19:00,Megastar,9773064,59.4466,24.7733,2.4,19,141,EETLL,06/04/2018 04:30,06/04/2018 04:30,2018-04-06 04:18:17,FIHEL,2018-04-06 06:30:00,2018-04-06 06:23:00,NaT,2018-04-06 06:27:09,128,-2,4,124
2018-04-06 04:20:00,4383,2018-04-06 04:20:00,Megastar,9773064,59.4473,24.7738,2.6,27,98,EETLL,06/04/2018 04:30,06/04/2018 04:30,2018-04-06 04:18:17,FIHEL,2018-04-06 06:30:00,2018-04-06 06:24:00,NaT,2018-04-06 06:27:09,127,-2,3,124
2018-04-06 04:21:00,4385,2018-04-06 04:21:00,Megastar,9773064,59.4480,24.7749,4.5,44,59,EETLL,06/04/2018 04:30,06/04/2018 04:30,2018-04-06 04:18:17,FIHEL,2018-04-06 06:30:00,2018-04-06 06:25:00,NaT,2018-04-06 06:27:09,126,-2,2,124
2018-04-06 04:22:00,4387,2018-04-06 04:22:00,Megastar,9773064,59.4495,24.7772,8.9,25,28,EETLL,06/04/2018 04:30,06/04/2018 04:30,2018-04-06 04:18:17,FIHEL,2018-04-06 06:30:00,2018-04-06 06:26:00,NaT,2018-04-06 06:27:09,125,-2,1,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-15 07:25:00,912160,2019-03-15 07:25:00,Megastar,9773064,60.1456,24.9134,1.4,330,194,EETLL,15/03/2019 05:30,15/03/2019 05:30,2019-03-15 05:19:04,FIHEL,2019-03-15 07:30:00,2019-03-15 07:27:00,NaT,2019-03-15 07:29:11,4,0,2,2
2019-03-15 07:26:00,912163,2019-03-15 07:26:00,Megastar,9773064,60.1461,24.9135,2.3,17,201,EETLL,15/03/2019 05:30,15/03/2019 05:30,2019-03-15 05:19:04,FIHEL,2019-03-15 07:30:00,2019-03-15 07:28:00,NaT,2019-03-15 07:29:11,3,0,1,2
2019-03-15 07:27:00,912166,2019-03-15 07:27:00,Megastar,9773064,60.1468,24.9139,2.9,20,205,EETLL,15/03/2019 05:30,15/03/2019 05:30,2019-03-15 05:19:04,FIHEL,2019-03-15 07:30:00,2019-03-15 07:28:00,NaT,2019-03-15 07:29:11,2,0,1,1


In [ ]:
MegastarEstoniaDF['maximumJourneyDuration'].describe()

count    130025.000000
mean         24.086276
std         372.232203
min          -6.000000
25%           0.000000
50%           0.000000
75%           1.000000
max       15182.000000
Name: maximumJourneyDuration, dtype: float64

In [ ]:
MegastarEstoniaDF['newTimeToDestination'].describe()

count    130025.000000
mean         86.867056
std         372.093766
min           0.000000
25%          33.000000
50%          66.000000
75%         100.000000
max       15247.000000
Name: newTimeToDestination, dtype: float64

In [ ]:
MegastarEstoniaDF[(MegastarEstoniaDF["maximumJourneyDuration"] > 0)]["maximumJourneyDuration"].unique()

array([    2,     1,     3,     4,     5,     7,     8,    11,     6,
          10,    37,  1092,    12,  1084,     9,    29,    41,    43,
          15,   719,    25,  1079,    44,   780,   781,  1436,   356,
          57,    59,    58,   782,   779,   783,    60,   778,   784,
       15182,    13], dtype=int16)

In [ ]:
MegastarEstoniaDF[(MegastarEstoniaDF["maximumJourneyDuration"] == 60)]

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-11-15 17:22:00,567687,2018-11-15 17:22:00,Megastar,9773064,59.4516,24.7799,11.2,11,11,EETLL,15/11/2018 17:30,15/11/2018 17:30,2018-11-15 17:17:12,FIHEL,2018-11-15 18:30:00,2018-11-15 19:29:00,NaT,2018-11-15 19:30:11,128,60,1,127
2018-11-15 17:23:00,567690,2018-11-15 17:23:00,Megastar,9773064,59.4547,24.7802,13.9,357,356,EETLL,15/11/2018 17:30,15/11/2018 17:30,2018-11-15 17:17:12,FIHEL,2018-11-15 18:30:00,2018-11-15 19:29:00,NaT,2018-11-15 19:30:11,127,60,1,126
2018-11-15 17:23:00,567694,2018-11-15 17:23:00,Megastar,9773064,59.4578,24.7794,15.6,349,348,EETLL,15/11/2018 17:30,15/11/2018 17:30,2018-11-15 17:17:12,FIHEL,2018-11-15 18:30:00,2018-11-15 19:29:00,NaT,2018-11-15 19:30:11,127,60,1,126
2018-11-15 17:25:00,567698,2018-11-15 17:25:00,Megastar,9773064,59.4632,24.7771,17.8,346,346,EETLL,15/11/2018 17:30,15/11/2018 17:30,2018-11-15 17:17:12,FIHEL,2018-11-15 18:30:00,2018-11-15 19:29:00,NaT,2018-11-15 19:30:11,125,60,1,124
2018-11-15 17:26:00,567702,2018-11-15 17:26:00,Megastar,9773064,59.4682,24.7747,18.7,346,346,EETLL,15/11/2018 17:30,15/11/2018 17:30,2018-11-15 17:17:12,FIHEL,2018-11-15 18:30:00,2018-11-15 19:27:00,NaT,2018-11-15 19:30:11,124,60,3,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21 19:26:00,666269,2018-12-21 19:26:00,Megastar,9773064,60.1457,24.9167,1.9,222,154,EETLL,21/12/2018 17:30,21/12/2018 17:30,2018-12-21 17:17:01,FIHEL,2018-12-21 18:30:00,2018-12-21 19:28:00,NaT,2018-12-21 19:30:00,4,60,2,2
2018-12-21 19:27:00,666271,2018-12-21 19:27:00,Megastar,9773064,60.1458,24.9158,2.5,314,172,EETLL,21/12/2018 17:30,21/12/2018 17:30,2018-12-21 17:17:01,FIHEL,2018-12-21 18:30:00,2018-12-21 19:29:00,NaT,2018-12-21 19:30:00,3,60,1,2
2018-12-21 19:28:00,666273,2018-12-21 19:28:00,Megastar,9773064,60.1467,24.9149,4.2,344,191,EETLL,21/12/2018 17:30,21/12/2018 17:30,2018-12-21 17:17:01,FIHEL,2018-12-21 18:30:00,2018-12-21 19:29:00,NaT,2018-12-21 19:30:00,2,60,1,1


In [ ]:
cleanedDataEstonia = MegastarEstoniaDF[(MegastarEstoniaDF["maximumJourneyDuration"] < 61)& (MegastarEstoniaDF["hdg"] <= 360)]
cleanedDataEstonia.describe()

,id,imo,lat,long,sog,cog,hdg,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
count,127638.000000,127638.0,127638.000000,127638.000000,127638.000000,127638.000000,127638.000000,127638.000000,127638.000000,127638.000000,127638.000000
mean,461113.318996,9773064.0,59.808730,24.815444,20.166863,104.765908,112.206365,65.086769,1.690092,1.422468,63.865158
std,268639.725055,0.0,0.225081,0.075206,6.073105,146.359372,145.284590,37.979719,8.448277,3.383612,39.071431
min,4381.000000,9773064.0,59.444100,24.598400,0.000000,0.000000,0.000000,0.000000,-6.000000,-21.000000,-52.000000
25%,222445.000000,9773064.0,59.611100,24.752800,21.400000,11.000000,10.000000,32.000000,0.000000,0.000000,30.000000
50%,455491.500000,9773064.0,59.808850,24.816000,22.500000,18.000000,19.000000,65.000000,0.000000,1.000000,64.000000
75%,710902.250000,9773064.0,60.012300,24.896700,23.100000,339.000000,337.000000,98.000000,1.000000,3.000000,97.000000
max,912171.000000,9773064.0,60.148000,24.924000,28.900000,360.000000,359.000000,176.000000,60.000000,58.000000,135.000000


In [ ]:
cleanedDataEstoniaNormalArrival = cleanedDataEstonia[(cleanedDataEstonia["maximumJourneyDuration"] <= 0)]
cleanedDataEstoniaLateArrival = cleanedDataEstonia[(cleanedDataEstonia["maximumJourneyDuration"] > 30)]
cleanedDataEstoniaLateArrival

,id,updated,ship,imo,lat,long,sog,cog,hdg,depPort,etdSchedule,etd,atd,arrPort,etaSchedule,eta,ata,timeToDestination,newTimeToDestination,maximumJourneyDuration,etaError,forecastedTimeToDestination
updated,,,,,,,,,,,,,,,,,,,,,,
2018-07-26 19:07:00,294469,2018-07-26 19:07:00,Megastar,9773064,60.1480,24.9148,0.0,170,208,EETLL,26/07/2018 16:30,26/07/2018 16:30,NaN,FIHEL,2018-07-26 18:30:00,2018-07-26 18:30:00,2018-07-26 19:07:42,2018-07-26 19:07:42,0,37,37,-37
2018-07-31 10:19:00,303363,2018-07-31 10:19:00,Megastar,9773064,59.4459,24.7724,3.6,15,163,EETLL,31/07/2018 10:30,31/07/2018 10:30,2018-07-31 10:19:07,FIHEL,2018-07-31 12:30:00,2018-07-31 12:31:00,NaT,2018-07-31 13:11:11,172,41,40,132
2018-07-31 10:20:00,303366,2018-07-31 10:20:00,Megastar,9773064,59.4468,24.7726,2.0,9,122,EETLL,31/07/2018 10:30,31/07/2018 10:30,2018-07-31 10:19:07,FIHEL,2018-07-31 12:30:00,2018-07-31 12:31:00,NaT,2018-07-31 13:11:11,171,41,40,131
2018-07-31 10:21:00,303369,2018-07-31 10:21:00,Megastar,9773064,59.4473,24.7738,3.8,58,81,EETLL,31/07/2018 10:30,31/07/2018 10:30,2018-07-31 10:19:07,FIHEL,2018-07-31 12:30:00,2018-07-31 12:32:00,NaT,2018-07-31 13:11:11,170,41,39,131
2018-07-31 10:22:00,303373,2018-07-31 10:22:00,Megastar,9773064,59.4480,24.7761,6.3,56,58,EETLL,31/07/2018 10:30,31/07/2018 10:30,2018-07-31 10:19:07,FIHEL,2018-07-31 12:30:00,2018-07-31 12:32:00,NaT,2018-07-31 13:11:11,169,41,39,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21 19:26:00,666269,2018-12-21 19:26:00,Megastar,9773064,60.1457,24.9167,1.9,222,154,EETLL,21/12/2018 17:30,21/12/2018 17:30,2018-12-21 17:17:01,FIHEL,2018-12-21 18:30:00,2018-12-21 19:28:00,NaT,2018-12-21 19:30:00,4,60,2,2
2018-12-21 19:27:00,666271,2018-12-21 19:27:00,Megastar,9773064,60.1458,24.9158,2.5,314,172,EETLL,21/12/2018 17:30,21/12/2018 17:30,2018-12-21 17:17:01,FIHEL,2018-12-21 18:30:00,2018-12-21 19:29:00,NaT,2018-12-21 19:30:00,3,60,1,2
2018-12-21 19:28:00,666273,2018-12-21 19:28:00,Megastar,9773064,60.1467,24.9149,4.2,344,191,EETLL,21/12/2018 17:30,21/12/2018 17:30,2018-12-21 17:17:01,FIHEL,2018-12-21 18:30:00,2018-12-21 19:29:00,NaT,2018-12-21 19:30:00,2,60,1,1


In [ ]:
cleanedDataEstonia[(cleanedDataEstonia["etaError"] > 0)]["etaError"].unique()

array([ 4,  3,  2,  1,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 37,
       29, 28, 27, 26, 25, 40, 39, 38, 41, 42, 43, 44, 45, 46, 17, 18, 22,
       21, 23, 24, 47, 48, 49, 50, 51, 52, 55, 53, 36, 58], dtype=int16)

In [ ]:
etaErrorCountEst = len(cleanedDataEstonia[(cleanedDataEstonia["etaError"] >= 5)].index)
totalCleanDataCountEst = len(cleanedDataEstonia.index)

import plotly.graph_objects as go

labels = ['Falsely Predicted_ETA','Total']
values = [etaErrorCountEst, totalCleanDataCountEst]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [ ]:
cleanedDataEstonia = cleanedDataEstonia[~cleanedDataEstonia.index.duplicated(keep='first')]
x_est = cleanedDataEstonia[["lat", "long", "sog", "cog", "hdg","forecastedTimeToDestination","newTimeToDestination"]].reset_index(drop=True)
x_est = x_est.astype({'lat': 'float32', 'long': 'float32', 'sog': 'float32', 'cog': 'float32','hdg': 'int32', 'newTimeToDestination': 'int32'})
x_est.dtypes

lat                            float32
long                           float32
sog                            float32
cog                            float32
hdg                              int32
forecastedTimeToDestination      int16
newTimeToDestination             int32
dtype: object

In [ ]:
X = x_est[["lat", "long", "sog", "cog", "hdg"]]
y = x_est["newTimeToDestination"]

from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

regressor_est = RandomForestRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [ ]:
regModel_est = regressor_est.fit(X_train,y_train)
#regModel.score(X_test,y_test)
regModel_est.score(X_train,y_train)

0.9991345992379255

In [ ]:
x_data = np.array(X)

keras_normalizer = preprocessing.Normalization(input_shape=[5,])
keras_normalizer.adapt(x_data)

In [ ]:
def build_model():
  model = keras.models.Sequential()
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dense(64, activation='relu',input_shape=(X_train.shape[1],)))
  model.add(keras.layers.Dense(64, activation='relu'))
  model.add(keras.layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=[coeff_determination])
  return model

model = build_model()
model.fit(X_train, y_train, epochs=50, batch_size=1000)

Epoch 1/50
83/83 [==============================] - 1s 3ms/step - loss: 5268.8510 - coeff_determination: -2.6477
Epoch 2/50
83/83 [==============================] - 0s 3ms/step - loss: 2089.9854 - coeff_determination: -0.4594
Epoch 3/50
83/83 [==============================] - 0s 3ms/step - loss: 196.6638 - coeff_determination: 0.8632
Epoch 4/50
83/83 [==============================] - 0s 3ms/step - loss: 44.5254 - coeff_determination: 0.9688
Epoch 5/50
83/83 [==============================] - 0s 3ms/step - loss: 22.1732 - coeff_determination: 0.9846
Epoch 6/50
83/83 [==============================] - 0s 3ms/step - loss: 18.0298 - coeff_determination: 0.9875
Epoch 7/50
83/83 [==============================] - 0s 3ms/step - loss: 16.4205 - coeff_determination: 0.9886
Epoch 8/50
83/83 [==============================] - 0s 3ms/step - loss: 15.4106 - coeff_determination: 0.9893
Epoch 9/50
83/83 [==============================] - 0s 3ms/step - loss: 16.7574 - coeff_determination: 0.9884
Epo

In [ ]:
regModel_est.score(X_test,y_test)

0.9932176374284873

In [ ]:
textDF = x_est.iloc[X_test.index]
textDF

,lat,long,sog,cog,hdg,forecastedTimeToDestination,newTimeToDestination
95444,60.101700,24.901100,17.200001,4.0,6,17,16
30962,60.110500,24.903799,16.400000,6.0,5,12,14
73967,59.559200,24.714399,22.799999,340.0,340,106,108
36553,60.133701,24.906700,16.600000,1.0,2,7,8
87100,59.446201,24.772400,5.600000,31.0,214,133,129
...,...,...,...,...,...,...,...
80868,59.448898,24.776100,6.400000,33.0,224,130,131
88028,59.499802,24.757200,23.100000,340.0,339,116,111
44378,59.894600,24.857300,22.299999,11.0,10,49,50
15104,59.842300,24.827801,23.100000,13.0,14,57,59


In [ ]:
from sklearn.metrics import r2_score
r2_score(textDF['newTimeToDestination'], textDF['forecastedTimeToDestination'])

0.9898145630908876

In [ ]:
testDataPrediction = regModel_est.predict(X_test)
testDataPrediction

array([ 16.77,  13.61, 106.62, ...,  50.6 ,  58.93,  25.51])

In [ ]:
neuralPrediction = model.predict(textDF[["lat", "long", "sog", "cog", "hdg"]])
neuralPrediction = [num[0] for num in neuralPrediction.tolist()]
neuralPrediction

[15.961471557617188,
 14.634687423706055,
 108.12442779541016,
 12.506771087646484,
 130.3538360595703,
 49.47147750854492,
 4.564352989196777,
 114.1380386352539,
 110.10283660888672,
 97.15888214111328,
 77.146484375,
 4.751723289489746,
 111.7743911743164,
 86.99324798583984,
 2.3728315830230713,
 33.88402557373047,
 22.948326110839844,
 102.11019897460938,
 117.54706573486328,
 61.81759262084961,
 69.68870544433594,
 50.3810920715332,
 50.929134368896484,
 119.4544906616211,
 117.44975280761719,
 20.247922897338867,
 54.76591110229492,
 32.53342056274414,
 29.27587127685547,
 15.46387004852295,
 79.41466522216797,
 99.94135284423828,
 3.1349050998687744,
 83.19197845458984,
 34.06454849243164,
 23.91423225402832,
 96.02972412109375,
 50.2070198059082,
 64.85704803466797,
 26.867055892944336,
 23.053998947143555,
 38.5353889465332,
 37.79985427856445,
 129.2694854736328,
 61.511783599853516,
 91.09864807128906,
 96.65784454345703,
 14.904414176940918,
 112.12517547607422,
 106.98860

In [ ]:
r2_score(y_test, neuralPrediction)

0.9925257902749338

In [ ]:
from sklearn.metrics import max_error
max_error(textDF['newTimeToDestination'], textDF['forecastedTimeToDestination'])

58

In [ ]:
max_error(y_test, testDataPrediction)

122.0

In [ ]:
max_error(y_test, neuralPrediction)

119.07133054733276

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(textDF['newTimeToDestination'], textDF['forecastedTimeToDestination'])

2.3466363011965443

In [ ]:
mean_absolute_error(y_test, testDataPrediction)

1.5056834537451607

In [ ]:
mean_absolute_error(y_test, neuralPrediction)

1.7894184348881685

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=neuralPrediction[0:50],
                    mode='lines',
                    name='Model Predicted ETA'))
fig.add_trace(go.Scatter(y=y_test[0:50],
                    mode='lines',
                    fill='tozeroy',
                    name='True ETA'))

fig.add_trace(go.Scatter(y=textDF['forecastedTimeToDestination'][0:50],
                    mode='lines',
                    name= 'Data Calculated ETA'))

fig.show()

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "tallinnTripETA")

In [ ]:
from tensorflow.keras.models import Model


In [ ]:
input_img = keras.Input(shape=(5,))

encoder = tf.keras.Sequential([
  keras.layers.BatchNormalization(),
  keras.layers.Dense(4, activation="relu"),
  keras.layers.Dense(2, activation="relu")])(input_img)

decoder = tf.keras.Sequential([
  keras.layers.Dense(2, activation="relu"),
  keras.layers.Dense(5)])(encoder)

autoencoder = keras.Model(input_img, decoder)
  
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
sequential_18 (Sequential)   (None, 2)                 54        
_________________________________________________________________
sequential_19 (Sequential)   (None, 5)                 21        
Total params: 75
Trainable params: 65
Non-trainable params: 10
_________________________________________________________________


In [ ]:
history = autoencoder.fit(X_train, X_train, 
          epochs=50, 
          batch_size=100,
          validation_data=(X_test, X_test),
          shuffle=True)

Epoch 1/50
830/830 [==============================] - 2s 2ms/step - loss: 61.1444 - val_loss: 41.0556
Epoch 2/50
830/830 [==============================] - 1s 1ms/step - loss: 41.2299 - val_loss: 40.2014
Epoch 3/50
830/830 [==============================] - 1s 1ms/step - loss: 38.0122 - val_loss: 7.0928
Epoch 4/50
830/830 [==============================] - 1s 1ms/step - loss: 7.8256 - val_loss: 5.7340
Epoch 5/50
830/830 [==============================] - 1s 1ms/step - loss: 7.5889 - val_loss: 6.4059
Epoch 6/50
830/830 [==============================] - 1s 1ms/step - loss: 7.6040 - val_loss: 5.9227
Epoch 7/50
830/830 [==============================] - 1s 1ms/step - loss: 7.2413 - val_loss: 5.6232
Epoch 8/50
830/830 [==============================] - 1s 1ms/step - loss: 7.4124 - val_loss: 5.7506
Epoch 9/50
830/830 [==============================] - 1s 1ms/step - loss: 7.1813 - val_loss: 5.3361
Epoch 10/50
830/830 [==============================] - 1s 1ms/step - loss: 7.2671 - val_loss: 5

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Threshold:  2.5071659276896954


In [ ]:
#reconstructions_est = autoencoder.predict(cleanedDataEstoniaLateArrival[["lat", "long", "sog", "cog", "hdg"]].reset_index(drop=True))
#test_loss = tf.keras.losses.mae(reconstructions_est, cleanedDataEstoniaLateArrival[["lat", "long", "sog", "cog", "hdg"]])
#threshold_est = np.mean(test_loss)
#threshold_est

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(autoencoder, "tallinnTripHealth")

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

#cleanedDataEstoniaNormalArrival = cleanedDataEstoniaNormalArrival[~cleanedDataEstoniaNormalArrival.index.duplicated(keep='first')]
#x_est = cleanedDataEstoniaNormalArrival[["lat", "long", "sog", "cog", "hdg"]].reset_index(drop=True)
clf = LocalOutlierFactor(novelty= True, contamination=0.001)
clf.fit(x_est[["lat", "long", "sog", "cog", "hdg"]])
x_est

In [ ]:
outlierResult = clf.predict(cleanedDataEstoniaLateArrival[["lat", "long", "sog", "cog", "hdg"]].reset_index(drop=True))
newlist = [item for item in outlierResult if item == -1]

print(len(newlist), len(outlierResult))

In [ ]:

#from sklearn import preprocessing
#scaler = preprocessing.MinMaxScaler();
#x1 = scaler.fit_transform(x)
#x = pd.DataFrame(x1, columns=["lat", "long", "sog", "cog", "hdg","newTimeToDestination"])


In [ ]:
#x2 = scaler.inverse_transform(x1)
#x2



1.   Data is not balanced, should try NOVELTY DETECTION or use python SMOGYN
2.   Check for overfitting

1.   Investigate the possibility of live training of data by batches

2.   Use Max error, MSE, to prove that my model is better than default ()
1.   Deploy the model to browser using onnx.js, then you can talk about edge computing during the presentation
2.   Issue https://github.com/tensorflow/tfjs/issues/4626

1.   Issue https://github.com/tensorflow/tfjs/issues/3786
2.   List item











In [ ]:
# Convert into ONNX format
!pip install skl2onnx

from skl2onnx import convert_sklearn, to_onnx
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('x', FloatTensorType([None, X_train.shape[1]]))]
onx = convert_sklearn(regModel, initial_types=initial_type)

with open("helsinkiJourneyLin.onnx", "wb") as f:
    f.write(onx.SerializeToString())
